In [ ]:
# meteostat 설치 필요 
# !pip install meteostat
import os
import pandas as pd
import requests
from io import StringIO
from requests.utils import requote_uri
from datetime import datetime
from meteostat import Point, Daily 

# 서울시 자치구 중심좌표 CSV 로드 (한글경로 자동 인코딩)
raw_url = (
    "https://raw.githubusercontent.com/cubensys/Korea_District/"
    "master/3_서울시_자치구/"
    "서울시_자치구_중심점_2017.csv"
)
url = requote_uri(raw_url)
resp = requests.get(url)
resp.encoding = "euc-kr"
df_loc = pd.read_csv(StringIO(resp.text), encoding="euc-kr")

# '시군구명', 'Y'(위도), 'X'(경도)에 맞춰 Point 객체로 매핑
locations = {
    row["시군구명"]: Point(row["Y"], row["X"])
    for _, row in df_loc.iterrows()
}

# 조회 기간 설정 
start = datetime(2023, 1, 1)
end   = datetime(2024, 12, 31)

# 결과 저장 폴더 생성
output_dir = "./data/seoul_weather"
os.makedirs(output_dir, exist_ok=True)

# 로케이션(Point)으로 직접 조회하여 CSV 저장
for gu, point in locations.items():
    # Point 객체를 직접 사용하여 일별 과거 데이터 조회
    data = Daily(point, start, end).fetch()
    
    # 데이터 확인
    print(f"{gu}: {len(data)} rows retrieved")
    
    # CSV로 저장
    filepath = os.path.join(output_dir, f"{gu}.csv")
    data.to_csv(filepath, encoding="utf-8-sig")
    print(f"Saved → {filepath}")